# Ricerca Sentinel 2 Level 2A con pySTAC Client

In [15]:
from pystac_client import Client
from odc.stac import load
import odc.geo
import folium

### Collection

In [6]:
client = Client.open("https://earth-search.aws.element84.com/v1")
collection = "sentinel-2-l2a"

### AOI

In [20]:
geometry = {
    "coordinates": [
        [
            [14.100242411640295,42.535115763088044],
            [14.070592054190996,42.53300920401685],
            [14.037319161231324,42.50968767009442],
            [14.067283770734832,42.46495526438207],
            [14.136523612765757,42.46703905494371],
            [14.161017079346948,42.5066823364991],
            [14.100242411640295,42.535115763088044]
        ]
    ],
    "type": "Polygon"
}

map = folium.Map()
polygon_coords = geometry["coordinates"][0]
bounds = [[point[1], point[0]] for point in polygon_coords]
polygon = folium.Polygon(locations=bounds, color='blue', fill=True).add_to(map)
map.fit_bounds(bounds)

map

### Search

In [12]:

date_range = "2024-05-01/2024-05-20"

# additional filters as per metadata 
filters = {
    "eo:cloud_cover":{"lt":0.2},
    "s2:vegetation_percentage": {"gt": 25}
}

# run pystac client search to see available dataset
search = client.search(
    collections=[collection], intersects=geometry, datetime=date_range
) 

# spit out data as GeoJSON dictionary
print(search.item_collection_as_dict())

# loop through each item
for item in search.items_as_dicts():
    print(item)

{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'stac_version': '1.0.0', 'id': 'S2B_33TVH_20240519_0_L2A', 'properties': {'created': '2024-05-19T15:13:48.488Z', 'platform': 'sentinel-2b', 'constellation': 'sentinel-2', 'instruments': ['msi'], 'eo:cloud_cover': 99.997497, 'proj:epsg': 32633, 'mgrs:utm_zone': 33, 'mgrs:latitude_band': 'T', 'mgrs:grid_square': 'VH', 'grid:code': 'MGRS-33TVH', 'view:sun_azimuth': 151.544996454177, 'view:sun_elevation': 64.7767716324504, 's2:degraded_msi_data_percentage': 0.0202, 's2:nodata_pixel_percentage': 2.138928, 's2:saturated_defective_pixel_percentage': 0, 's2:dark_features_percentage': 0, 's2:cloud_shadow_percentage': 0, 's2:vegetation_percentage': 0, 's2:not_vegetated_percentage': 0.000234, 's2:water_percentage': 0.002272, 's2:unclassified_percentage': 0, 's2:medium_proba_clouds_percentage': 45.653805, 's2:high_proba_clouds_percentage': 52.050841, 's2:thin_cirrus_percentage': 2.292851, 's2:snow_ice_percentage': 0, 's2:product_type':

### Get data from Items

In [14]:
#load the data in xarray format
data = load(search.items() ,geopolygon=geometry,groupby="solar_day", chunks={})
data

<xarray.Dataset> Size: 417MB
Dimensions:       (y: 777, x: 1017, time: 8)
Coordinates:
  * y             (y) float64 6kB 4.71e+06 4.71e+06 ... 4.702e+06 4.702e+06
  * x             (x) float64 8kB 4.209e+05 4.209e+05 ... 4.311e+05 4.311e+05
    spatial_ref   int32 4B 32633
  * time          (time) datetime64[ns] 64B 2024-05-01T09:58:56.057000 ... 20...
Data variables: (12/32)
    aot           (time, y, x) uint16 13MB dask.array<chunksize=(1, 777, 1017), meta=np.ndarray>
    blue          (time, y, x) uint16 13MB dask.array<chunksize=(1, 777, 1017), meta=np.ndarray>
    coastal       (time, y, x) uint16 13MB dask.array<chunksize=(1, 777, 1017), meta=np.ndarray>
    green         (time, y, x) uint16 13MB dask.array<chunksize=(1, 777, 1017), meta=np.ndarray>
    nir           (time, y, x) uint16 13MB dask.array<chunksize=(1, 777, 1017), meta=np.ndarray>
    nir08         (time, y, x) uint16 13MB dask.array<chunksize=(1, 777, 1017), meta=np.ndarray>
    ...            ...
    rededge3-jp2  (time, y, x) uint16 13MB dask.array<chunksize=(1, 777, 1017), meta=np.ndarray>
    scl-jp2       (time, y, x) uint8 6MB dask.array<chunksize=(1, 777, 1017), meta=np.ndarray>
    swir16-jp2    (time, y, x) uint16 13MB dask.array<chunksize=(1, 777, 1017), meta=np.ndarray>
    swir22-jp2    (time, y, x) uint16 13MB dask.array<chunksize=(1, 777, 1017), meta=np.ndarray>
    visual-jp2    (time, y, x) float32 25MB dask.array<chunksize=(1, 777, 1017), meta=np.ndarray>
    wvp-jp2       (time, y, x) uint16 13MB dask.array<chunksize=(1, 777, 1017), meta=np.ndarray>